In [25]:
import pandas as pd
import torch.nn as nn
import pickle
import torch
from torchvision import models
from torchvision.models import detection, resnet50, ResNet50_Weights
import os
import numpy as np
import cv2
from torchvision import transforms
import pymc3 as pm
import theano.tensor as tt
from sklearn.preprocessing import LabelEncoder
import scipy

In [2]:
CONFIGS = {
    # determine the current device and based on that set the pin memory
    # flag
    "DEVICE": "cuda" if torch.cuda.is_available() else "cpu",
    # specify ImageNet mean and standard deviation
    "IMG_MEAN": [0.485, 0.456, 0.406],
    "IMG_STD": [0.229, 0.224, 0.225],
    "MC_DROPOUT_ENABLED": False,  # Switch to enable/disable MC Dropout for confidence score
    "NUM_DROPOUT_RUNS": 3,
    "CONFIDENCE_THRESHOLD": 0,
    "BIG_MODEL_IMG_SIZE": 320,
    "SMALL_MODEL_IMG_SIZE": 60,
    "MEAN_PRIOR": -15,
}

# Big model

## Model loading

In [3]:
class MultiHeadResNet_BigModel(nn.Module):
    def __init__(self, num_classes_prdtype, num_classes_weight, num_classes_halal, num_classes_healthy):
        super(MultiHeadResNet_BigModel, self).__init__()
        self.base_model = models.resnet50(weights=ResNet50_Weights.DEFAULT)
        num_ftrs = self.base_model.fc.in_features
        self.base_model.fc = nn.Identity()

        # Define custom fully connected layers for each prediction head
        self.fc_prdtype = nn.Linear(num_ftrs, num_classes_prdtype)
        self.fc_weight = nn.Linear(num_ftrs, num_classes_weight)
        self.fc_halal = nn.Linear(num_ftrs, num_classes_halal)
        self.fc_healthy = nn.Linear(num_ftrs, num_classes_healthy)
        self.fc_bbox = nn.Linear(num_ftrs, 4)

    def forward(self, x):
        x = self.base_model(x)
        prdtype = self.fc_prdtype(x)
        weight = self.fc_weight(x)
        halal = self.fc_halal(x)
        healthy = self.fc_healthy(x)
        box = self.fc_bbox(x)
        return prdtype, weight, halal, healthy, box

    
# load label encoder 
def load_label_encoder_big_model():
    le_prdtype = pickle.loads(open("NN_model/regularized/le_prdtype.pickle", "rb").read())
    le_weight = pickle.loads(open("NN_model/regularized/le_weight.pickle", "rb").read())
    le_halal = pickle.loads(open("NN_model/regularized/le_halal.pickle", "rb").read())
    le_healthy = pickle.loads(open("NN_model/regularized/le_healthy.pickle", "rb").read())
    
    return le_prdtype, le_weight, le_halal, le_healthy

le_prdtype, le_weight, le_halal, le_healthy = load_label_encoder_big_model()

# Load the trained MultiHeadResNet model
def load_model():
    # Verify the number of classes for each label
    num_classes_prdtype = len(le_prdtype.classes_)
    num_classes_weight = len(le_weight.classes_)
    num_classes_halal = len(le_halal.classes_)
    num_classes_healthy = len(le_healthy.classes_)
    # print(num_classes_prdtype)
    # print(num_classes_healthy)

    custom_resnet_model = MultiHeadResNet_BigModel(
        num_classes_prdtype=num_classes_prdtype,
        num_classes_weight=num_classes_weight,
        num_classes_halal=num_classes_halal,
        num_classes_healthy=num_classes_healthy
    )

    model_path = 'NN_model/regularized/multi_head_model.pth'
    # print("test1")
    if os.path.exists(model_path):
        custom_resnet_model.load_state_dict(torch.load(model_path, map_location=CONFIGS['DEVICE']))
    else:
        raise FileNotFoundError(f"Model file not found: {model_path}")
    # print("test2")
    custom_resnet_model.to(CONFIGS['DEVICE'])
    custom_resnet_model.eval()
    return custom_resnet_model

big_model = load_model()

/Users/liupeng/opt/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator LabelEncoder from version 1.2.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,


## Scoring on main imgs

In [4]:
main_imgs_results_big_model = pd.read_csv("NN_model/regularized/main_imgs_results_big_model.csv")
main_imgs_results_big_model.head()

,Filename,CorrectTotalLabel,ProductType_AdultMilk,ProductType_BabyMilkPowder,ProductType_Babyfood,ProductType_BeeHoonVermicelli,ProductType_BeeHoonVermicelliMeeSua,ProductType_BeehoonVermicelliMeesua,ProductType_BiscuitCrackersCookies,ProductType_BiscuitsCrackersCookies,...,Weight_400-499g,Weight_500-599g,Weight_600-699g,Weight_700-799g,Weight_800-899g,Weight_900-999g,HalalStatus_Halal,HalalStatus_NonHalal,HealthStatus_Healthy,HealthStatus_NonHealthy
0,IMG_20230428_124008_jpg.rf.899f0412923b03c8752...,Salt_500-599g_NonHalal_NonHealthy,-2.988563,-1.384374,-2.801754,-2.635914,-2.956713,-2.413638,-1.236322,-1.738432,...,-1.179443,4.841335,-1.014125,-0.808293,-1.010904,-0.779902,-0.256162,1.208414,-0.688717,0.887579
1,IMG_5674-1-_jpeg.rf.b36d574fd9f3c1cbff03debcf8...,InstantNoodles_400-499g_Halal_NonHealthy,-1.517324,-1.477158,-1.799352,-1.775014,-1.375650,-1.634192,-1.762195,-0.884902,...,4.468603,-0.725103,-0.684793,-0.695387,-1.024980,-0.629591,0.979091,-0.391913,-0.541426,1.030857
2,output_frame_0368.png,Babyfood_200-299g_Halal_NonHealthy,-1.837415,-1.328996,6.974995,-2.060615,-2.123350,-1.484981,-2.227858,-1.816216,...,-1.176241,-0.827538,-0.587794,-0.953934,-0.784789,-0.699471,1.277705,-0.522688,-0.673875,0.896023
3,IMG_5576_jpeg.rf.62ab26ef9618e74ccccfd5c60f4f7...,BiscuitsCrackersCookies_400-499g_Halal_NonHealthy,-2.394365,-1.373971,-2.485732,-2.016415,-2.443543,-2.391535,-2.412420,6.884818,...,4.401903,-1.431951,-0.880475,-0.684266,-1.354441,-0.989935,1.377815,-0.731200,-0.520227,0.960514
4,20240123_2_2699.jpg,OtherHotBeverages(Powder)_0-99g_Halal_NonHealthy,-3.175072,-2.886428,-3.853138,-2.901442,-2.690182,-2.472945,-3.292425,-0.606029,...,-0.865978,-1.067099,-1.769888,-0.323548,-1.011899,-0.887248,1.561953,-0.365296,-0.387846,0.917203


In [5]:
# Create a copy of the current column names to a list
new_columns = main_imgs_results_big_model.columns.tolist()

# Modify the first two elements
new_columns[0] = 'filepath'
new_columns[1] = 'label'

# Assign the modified list of column names back to the DataFrame
main_imgs_results_big_model.columns = new_columns


## Scoring on new imgs

In [6]:
new_imgs_df = pd.read_csv("GPT_model/chatgpt_prediction.csv")
new_imgs_df.head()

,Unnamed: 0,img_filename,product_type,image_reflection,image_clarity,prediction_confidence
0,0,IMG_0184_jpeg.rf.446d62343e17b8ef909b6f0ff4530...,OtherNoodles,Medium,High,Low
1,1,IMG_0118_jpeg.rf.305bb0eadb135b869c11332c7173a...,OtherSauceDressingSoupbasePaste,Low,High,High
2,2,IMG_0096_jpeg.rf.4b1ab7e2cd4c285bd9e74d1ec1ab0...,Book,Low,Medium,Medium
3,3,IMG_0105_jpeg.rf.62a7a2c99f2d635525abcca89cb9f...,OtherSauceDressingSoupbasePaste,Low,Medium,Low
4,4,IMG_0124_jpeg.rf.3d66e8ddcd7d9d088875a3e2cf365...,OtherDriedFood,Low,High,High


In [7]:
transforms_test = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize(mean=CONFIGS['IMG_MEAN'], std=CONFIGS['IMG_STD'])
])

In [53]:
master_df = pd.read_csv("../../master_list.csv")
master_df.head()

,filepath,xmin,ymin,xmax,ymax,label,ProductType,Weight,HalalStatus,HealthStatus,new_camera,tag
0,IMG_20230428_123528_jpg.rf.5687b7b914f6d9aa98c...,151,42,497,591,Sugar_400-499g_NonHalal_NonHealthy,Sugar,400-499g,NonHalal,NonHealthy,0,NaN
1,IMG_20230428_123522_jpg.rf.204ff37f497f2dce442...,88,81,442,567,Sugar_400-499g_NonHalal_NonHealthy,Sugar,400-499g,NonHalal,NonHealthy,0,NaN
2,IMG_20230428_123708_jpg.rf.141ecd0cefaea75c0b7...,35,34,492,622,Sugar_400-499g_NonHalal_NonHealthy,Sugar,400-499g,NonHalal,NonHealthy,0,NaN
3,IMG_20230428_123521_jpg.rf.1069b402272252862ec...,99,122,428,587,Sugar_400-499g_NonHalal_NonHealthy,Sugar,400-499g,NonHalal,NonHealthy,0,NaN
4,IMG_20230428_123659_jpg.rf.5e1b6c4caabe48cf360...,103,17,474,592,Sugar_400-499g_NonHalal_NonHealthy,Sugar,400-499g,NonHalal,NonHealthy,0,NaN


In [56]:
np.unique(master_df.HealthStatus)

array(['Healthy', 'NonHealthy'], dtype=object)

In [24]:
new_imgs_results_big_model = []  # List to store the results

for idx, row in new_imgs_df.iterrows():
    image_path = "../../all_images/" + row['img_filename']
    frame = cv2.imread(image_path)

    # Preprocessing steps
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, (CONFIGS['BIG_MODEL_IMG_SIZE'], CONFIGS['BIG_MODEL_IMG_SIZE']))
    frame = frame.transpose((2, 0, 1))
    frame = torch.from_numpy(frame).float()
    frame = transforms_test(frame).unsqueeze(0).to(CONFIGS['DEVICE'])

    # Perform prediction
    with torch.no_grad():
        out1, out2, out3, out4, _ = big_model(frame)

    # reference the correct label from master list
    tmp_correct_label = master_df.loc[master_df['filepath'] == row['img_filename'], 'label'].iloc[0]
        
    # Extract and store the results
    prediction_row = [row['img_filename'], tmp_correct_label]
    prediction_row.extend(out1.cpu().numpy().flatten())
    prediction_row.extend(out2.cpu().numpy().flatten())
    prediction_row.extend(out3.cpu().numpy().flatten())
    prediction_row.extend(out4.cpu().numpy().flatten())
    new_imgs_results_big_model.append(prediction_row)


# Define column names for the new DataFrame
column_names = ['filepath', 'label']
big_model_pred_col_name_original = main_imgs_results_big_model.columns[2:].tolist()
column_names += big_model_pred_col_name_original

# Create the DataFrame
new_imgs_results_big_model = pd.DataFrame(new_imgs_results_big_model, columns=column_names)
new_imgs_results_big_model.head()

KeyboardInterrupt: 

In [10]:
new_imgs_results_big_model.shape

(350, 108)

## All scorings from big model

In [11]:
main_imgs_results_big_model['img_type'] = "existing"
new_imgs_results_big_model['img_type'] = "new"
all_imgs_results_big_model = pd.concat([main_imgs_results_big_model, new_imgs_results_big_model], axis=0)
all_imgs_results_big_model.reset_index(drop=True, inplace=True)
all_imgs_results_big_model.head()

,filepath,label,ProductType_AdultMilk,ProductType_BabyMilkPowder,ProductType_Babyfood,ProductType_BeeHoonVermicelli,ProductType_BeeHoonVermicelliMeeSua,ProductType_BeehoonVermicelliMeesua,ProductType_BiscuitCrackersCookies,ProductType_BiscuitsCrackersCookies,...,Weight_500-599g,Weight_600-699g,Weight_700-799g,Weight_800-899g,Weight_900-999g,HalalStatus_Halal,HalalStatus_NonHalal,HealthStatus_Healthy,HealthStatus_NonHealthy,img_type
0,IMG_20230428_124008_jpg.rf.899f0412923b03c8752...,Salt_500-599g_NonHalal_NonHealthy,-2.988563,-1.384374,-2.801754,-2.635914,-2.956713,-2.413638,-1.236322,-1.738432,...,4.841335,-1.014125,-0.808293,-1.010904,-0.779902,-0.256162,1.208414,-0.688717,0.887579,existing
1,IMG_5674-1-_jpeg.rf.b36d574fd9f3c1cbff03debcf8...,InstantNoodles_400-499g_Halal_NonHealthy,-1.517324,-1.477158,-1.799352,-1.775014,-1.375650,-1.634192,-1.762195,-0.884902,...,-0.725103,-0.684793,-0.695387,-1.024980,-0.629591,0.979091,-0.391913,-0.541426,1.030857,existing
2,output_frame_0368.png,Babyfood_200-299g_Halal_NonHealthy,-1.837415,-1.328996,6.974995,-2.060615,-2.123350,-1.484981,-2.227858,-1.816216,...,-0.827538,-0.587794,-0.953934,-0.784789,-0.699471,1.277705,-0.522688,-0.673875,0.896023,existing
3,IMG_5576_jpeg.rf.62ab26ef9618e74ccccfd5c60f4f7...,BiscuitsCrackersCookies_400-499g_Halal_NonHealthy,-2.394365,-1.373971,-2.485732,-2.016415,-2.443543,-2.391535,-2.412420,6.884818,...,-1.431951,-0.880475,-0.684266,-1.354441,-0.989935,1.377815,-0.731200,-0.520227,0.960514,existing
4,20240123_2_2699.jpg,OtherHotBeverages(Powder)_0-99g_Halal_NonHealthy,-3.175072,-2.886428,-3.853138,-2.901442,-2.690182,-2.472945,-3.292425,-0.606029,...,-1.067099,-1.769888,-0.323548,-1.011899,-0.887248,1.561953,-0.365296,-0.387846,0.917203,existing


In [12]:
all_imgs_results_big_model.tail()

,filepath,label,ProductType_AdultMilk,ProductType_BabyMilkPowder,ProductType_Babyfood,ProductType_BeeHoonVermicelli,ProductType_BeeHoonVermicelliMeeSua,ProductType_BeehoonVermicelliMeesua,ProductType_BiscuitCrackersCookies,ProductType_BiscuitsCrackersCookies,...,Weight_500-599g,Weight_600-699g,Weight_700-799g,Weight_800-899g,Weight_900-999g,HalalStatus_Halal,HalalStatus_NonHalal,HealthStatus_Healthy,HealthStatus_NonHealthy,img_type
4458,IMG_3392_jpeg.rf.84e82815866cb1cdd4b4fb22976f5...,Nuts_100-199g_Halal_Healthy,-1.509336,-1.421079,-1.467794,-2.305103,-2.504633,-2.051327,-1.888456,-0.630790,...,0.182243,-1.037051,-1.168795,-0.638293,-0.816123,0.882408,0.100910,-0.778074,0.704915,new
4459,IMG_3391_jpeg.rf.cdb42e0a2853b6d84ec8225930c83...,Nuts_100-199g_Halal_Healthy,-1.368625,-1.232493,-2.273168,-2.153253,-1.882854,-1.470379,-1.834718,-0.083466,...,-0.180690,-0.514829,-0.722491,-0.268586,-0.759063,0.440567,0.240890,-1.047939,0.744046,new
4460,IMG_3379_jpeg.rf.efb5f19e78349200e1c9120192b5c...,Nuts_100-199g_Halal_Healthy,-1.880809,-2.316740,-1.316481,-1.898985,-1.779359,-1.532286,-1.986148,-0.876322,...,-1.092089,-0.293497,-0.814427,-0.946109,-0.636806,0.922215,-0.020777,-1.072101,0.602360,new
4461,IMG_3368_jpeg.rf.f8dfe3e00309e5beee240208f5977...,Nuts_100-199g_Halal_Healthy,-2.117502,-2.000150,-0.960471,-1.695831,-2.389875,-1.281340,-2.203254,-2.466754,...,-1.273629,-0.915271,-1.090568,-0.906112,-0.404584,1.158328,-0.241587,-0.777947,0.161988,new
4462,IMG_3380_jpeg.rf.fd566d9f933b40636da05831f41d1...,Nuts_100-199g_Halal_Healthy,-1.978492,-1.930619,-1.521460,-1.978950,-2.149937,-1.614066,-1.763998,-0.918609,...,-1.247237,-0.696747,-0.853492,-0.574891,-0.739006,1.288738,-0.444384,-0.698532,0.302433,new


In [13]:
all_imgs_results_big_model.shape

(4463, 109)

In [14]:
all_imgs_results_big_model.to_csv("NN_model/regularized/all_imgs_results_big_model.csv", index=True)

In [15]:
class MultiHeadResNet_SmallModel(nn.Module):
    def __init__(self, num_classes_prdtype, num_classes_weight, num_classes_halal, num_classes_healthy):
        super(MultiHeadResNet_SmallModel, self).__init__()
        self.base_model = models.resnet18(pretrained=True)
        num_ftrs = self.base_model.fc.in_features
        self.base_model.fc = nn.Identity()

        # Define custom fully connected layers for each prediction head
        self.fc_prdtype = nn.Linear(num_ftrs, num_classes_prdtype)
        self.fc_weight = nn.Linear(num_ftrs, num_classes_weight)
        self.fc_halal = nn.Linear(num_ftrs, num_classes_halal)
        self.fc_healthy = nn.Linear(num_ftrs, num_classes_healthy)

    def forward(self, x):
        x = self.base_model(x)
        prdtype = self.fc_prdtype(x)
        weight = self.fc_weight(x)
        halal = self.fc_halal(x)
        healthy = self.fc_healthy(x)
        return prdtype, weight, halal, healthy

    
# load label encoder 
def load_label_encoder_small_model():
    le_prdtype = pickle.loads(open("../small_model/output/le_prdtype.pickle", "rb").read())
    le_weight = pickle.loads(open("../small_model/output/le_weight.pickle", "rb").read())
    le_halal = pickle.loads(open("../small_model/output/le_halal.pickle", "rb").read())
    le_healthy = pickle.loads(open("../small_model/output/le_healthy.pickle", "rb").read())
    
    return le_prdtype, le_weight, le_halal, le_healthy

le_prdtype, le_weight, le_halal, le_healthy = load_label_encoder_small_model()

# Load the trained MultiHeadResNet model
def load_model():
    # Verify the number of classes for each label
    num_classes_prdtype = len(le_prdtype.classes_)
    num_classes_weight = len(le_weight.classes_)
    num_classes_halal = len(le_halal.classes_)
    num_classes_healthy = len(le_healthy.classes_)
    # print(num_classes_prdtype)
    # print(num_classes_healthy)

    custom_resnet_model = MultiHeadResNet_SmallModel(
        num_classes_prdtype=num_classes_prdtype,
        num_classes_weight=num_classes_weight,
        num_classes_halal=num_classes_halal,
        num_classes_healthy=num_classes_healthy
    )

    model_path = '../small_model/output/multi_head_model.pth'
    # print("test1")
    if os.path.exists(model_path):
        custom_resnet_model.load_state_dict(torch.load(model_path, map_location=CONFIGS['DEVICE']))
    else:
        raise FileNotFoundError(f"Model file not found: {model_path}")
    # print("test2")
    custom_resnet_model.to(CONFIGS['DEVICE'])
    custom_resnet_model.eval()
    return custom_resnet_model
 
small_model = load_model()

FileNotFoundError: [Errno 2] No such file or directory: '../small_model/output/le_prdtype.pickle'

In [ ]:
new_imgs_df = pd.read_csv("../small_model/new_imgs_list.csv")
new_imgs_df.reset_index(drop=True, inplace=True)

# ADHOC: change the new imgs to existing type
new_imgs_df['label'] = 'AdultMilk_1-99g_Halal_NonHealthy'
new_imgs_df['ProductType'] = 'AdultMilk'
new_imgs_df['Weight'] = '1-99g'
new_imgs_df['HalalStatus'] = 'Halal'
new_imgs_df['HealthStatus'] = 'NonHealthy'

new_imgs_df.head()

In [ ]:
new_imgs_results_small_model = pd.read_csv("../small_model/new_imgs_results_small_model.csv")
new_imgs_results_small_model = new_imgs_results_small_model.loc[new_imgs_results_small_model.Filename.isin(new_imgs_df.filepath)]
new_imgs_results_small_model.reset_index(drop=True, inplace=True)
new_imgs_results_small_model.head()

In [ ]:
new_imgs_results_small_model.shape

In [ ]:
# Create a copy of the current column names to a list
new_columns = new_imgs_results_small_model.columns.tolist()

# Modify the first two elements
new_columns[0] = 'filepath'
new_columns[1] = 'label'

# Assign the modified list of column names back to the DataFrame
new_imgs_results_small_model.columns = new_columns
new_imgs_results_small_model.head()

In [ ]:
# Check if any name from 'extracted_names' is not in 'df' and add it as a new column
new_prdtype = list(set(all_imgs_results_big_model.columns) - set(new_imgs_results_small_model.columns))

if len(new_prdtype)>0:
    for col in new_prdtype:
        new_imgs_results_small_model[col] = np.random.normal(loc=CONFIGS["MEAN_PRIOR"], scale=np.sqrt(0.1), size=new_imgs_results_small_model.shape[0])  # Initialize new columns

new_imgs_results_small_model.head()  # Display the updated DataFrame for verificatio

In [ ]:
new_imgs_results_small_model.shape

In [ ]:
main_imgs_master_list = pd.read_csv("../master_list.csv")
main_imgs_master_list.head()

In [ ]:
main_imgs_results_small_model = []  # List to store the results
le_prdtype, le_weight, le_halal, le_healthy = load_label_encoder_small_model()

for idx, row in main_imgs_master_list.iterrows():
    image_path = "../all_images/" + row['filepath']
    frame = cv2.imread(image_path)

    # Preprocessing steps
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, (CONFIGS['SMALL_MODEL_IMG_SIZE'], CONFIGS['SMALL_MODEL_IMG_SIZE']))
    frame = frame.transpose((2, 0, 1))
    frame = torch.from_numpy(frame).float()
    frame = transforms_test(frame).unsqueeze(0).to(CONFIGS['DEVICE'])

    # Perform prediction
    with torch.no_grad():
        out1, out2, out3, out4 = small_model(frame)
    
    # Extract and store the results
    prediction_row = [row['filepath'], row['label']]
    prediction_row.extend(out1.cpu().numpy().flatten())
    prediction_row.extend(out2.cpu().numpy().flatten())
    prediction_row.extend(out3.cpu().numpy().flatten())
    prediction_row.extend(out4.cpu().numpy().flatten())
    main_imgs_results_small_model.append(prediction_row)


# Define column names for the new DataFrame
column_names = ['filepath', 'label']
column_names += ['ProductType_' + name for name in le_prdtype.classes_]
column_names += ['Weight_' + name for name in le_weight.classes_]
column_names += ['HalalStatus_' + name for name in le_halal.classes_]
column_names += ['HealthStatus_' + name for name in le_healthy.classes_]


# Create the DataFrame
main_imgs_results_small_model = pd.DataFrame(main_imgs_results_small_model, columns=column_names)
main_imgs_results_small_model.head()

In [ ]:
main_imgs_results_small_model.shape

In [ ]:
# Check if any name from 'extracted_names' is not in 'df' and add it as a new column
new_prdtype = list(set(all_imgs_results_big_model.columns) - set(main_imgs_results_small_model.columns))

if len(new_prdtype)>0:
    for col in new_prdtype:
        main_imgs_results_small_model[col] = np.random.normal(loc=CONFIGS["MEAN_PRIOR"], scale=np.sqrt(0.1), size=main_imgs_results_small_model.shape[0])  # Initialize new columns

main_imgs_results_small_model.head()  # Display the updated DataFrame for verificatio

In [ ]:
main_imgs_results_small_model.shape

In [ ]:
main_imgs_results_small_model['img_type'] = "existing"
new_imgs_results_small_model['img_type'] = "new"
all_imgs_results_small_model = pd.concat([main_imgs_results_small_model, new_imgs_results_small_model], axis=0)
all_imgs_results_small_model.reset_index(drop=True, inplace=True)
all_imgs_results_small_model.head()

In [ ]:
all_imgs_results_small_model.tail()

In [ ]:
all_imgs_results_small_model.shape

In [ ]:
all_imgs_results_small_model.to_csv("all_imgs_results_small_model.csv", index=True)

array(['AdultMilk', 'BabyMilkPowder', 'Babyfood', 'BeeHoonVermicelli',
       'BeeHoonVermicelliMeeSua', 'BeehoonVermicelliMeesua',
       'BiscuitCrackersCookies', 'BiscuitsCrackersCookies', 'Book',
       'BreakfastCereals', 'BreakfastCerealsCornflakes',
       'CannedBakedBeans', 'CannedBeefOtherMeats', 'CannedBraisedPeanuts',
       'CannedChicken', 'CannedFruitsCocktail', 'CannedMashrooms',
       'CannedPacketCreamersSweet', 'CannedPickles',
       'CannedPorkLunchronMeat', 'CannedSardingMackerel', 'CannedSoup',
       'CannedTunaDace', 'CannedVegetarianFood', 'CerealBeveragePowder',
       'ChilliSauce', 'ChocolateMaltPowder', 'Coffee', 'Coffee(Powder)',
       'CookingCreamMilk', 'CornChip', 'CurryPastePowderSauce',
       'DarkSoySauce', 'DriedBeans(RedGreenSdy)', 'DriedFood',
       'DriedFruits', 'DriedMeatSeafood', 'DriedVegetables',
       'FlavoredMilk', 'Flour', 'FruitJuice', 'FruitJuiceDrink',
       'HeabsSpicesPepper', 'HoneyOtherSpreads', 'InstantMeals',
       'Inst